# **This notebooks is used to plot a panel dashboard to visualize data for the GFTS project**  
____
The [panel dashboard](https://panel.holoviz.org/) displays the following informations :
- The temperature measured by the fish.
- The depth measured by the fish.
- The evolution of the latitude and the longitude during the time.
___
First of all, you need to access the following [jubyterhub server](https://gfts.minrk.net), you will need to log with your github account and you will select a configuration for starting a notebook. 

To acces this notebook you have to clone the following [github repository](https://github.com/destination-earth/DestinE_ESA_GFTS). 
First you will fork the repository, see this [link](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo). 
Then, you will clone **your forked version**, you will use the git jupyter extension, the section is in the sidebar. Here is a [tutorial](https://blog.reviewnb.com/jupyterlab-git-extension/#git-workflow-clone).  
If you want to submit code reviews, please see this [rule of participation](https://github.com/destination-earth/DestinE_ESA_GFTS/blob/main/docs/rule_of_participation.md)

To be able to run this notebook you need to clone [pangeo-fish](https://github.com/IAOCEA/pangeo-fish) repository, you can reuse the previous steps you used before, no need to fork the repository this time. 
Once you have clone the pangeo-fish repository, you need to install it with the following command :  
`!pip install pangeo-fish/`  /!\ You might need to update the path to the pangeo-fish folder you cloned.  
Normally, you can turn the first cell of the notebook into a code cell (select the cell and press Y key) and run it, wait for the installation to finish, then turn it back into a raw cell (select the cell and press R key), otherwise this cell will needlessly be runned by panel once you launch the dashboard. You can also use the notebook toolbar to change the cell type form code to raw.

Normally, all the other library to use this notebook should already installed once this has been done. To start the preview with panel, click the blue pannel logo Panel logo in the notebook tool bar.  
This will open a new tab in your notebook that displays panel informations.

Please note that all the tracks have been generated but note checked manually, some can display incoherent data. It will be checked and corrected later on.

In [ ]:
# Import necessary libraries and modules.
import os
import xarray as xr
from pangeo_fish.io import open_tag
import pandas as pd
import movingpandas as mpd
from pangeo_fish.tags import to_time_slice
import holoviews as hv
import panel as pn
from pangeo_fish.io import read_trajectories
from pangeo_fish import visualization
import s3fs

In [ ]:
### Parameters to access data, no need to modify to load tags normally

pn.extension()
s3 = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
    },
)
# Tag list is the list of available tags
tag_list_ = s3.ls("gfts-ifremer/tags/bargip/tracks")
tag_list = [
    tag.replace("gfts-ifremer/tags/bargip/tracks/", "")
    for tag in tag_list_
    if tag.replace("gfts-ifremer/tags/bargip/tracks/", "")
    in [
        fail.replace(".ipynb", "")
        for fail in os.listdir("../notebooks/papermill/papermill_output/done/")
    ]
]

cloud_root = "s3://gfts-ifremer/tags/bargip"

# tag_root specifies the root URL for tag data used for this computation.
tag_root = f"{cloud_root}/cleaned"

# scratch_root specifies the root directory where are GFTS computation data stored.
scratch_root = f"{cloud_root}/tracks"

# storage_options specifies options for the filesystem storing and/or opening output files.
storage_options = {
    "anon": False,
    # 'profile' : "gfts",
    "client_kwargs": {
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    },
}

# bbox, bounding box, defines the latitude and longitude range for the analysis area.
bbox = {"latitude": [42, 53], "longitude": [-8, 4]}
# tramodes are the two types of track that have been computed for GFTS.
track_modes = ["mean", "mode"]

In [ ]:
# pn.cache stores all the plot outputs to avoid doing the computation every time. Might need to disable if manipulating a wide amount of files.
@pn.cache

# Functions to plot the different visualization for a given tag id
def plot_time_series(plot_type="time series", tag_id="CB_A11071"):
    # load trajectories
    trajectories = read_trajectories(
        track_modes, f"{scratch_root}/{tag_id}", storage_options, format="parquet"
    )

    # Converting the trajectories to pandas DataFrames to access data easily
    mean_df = trajectories.trajectories[0].df
    mode_df = trajectories.trajectories[1].df

    tag = open_tag(tag_root, tag_id)
    time_slice = to_time_slice(tag["tagging_events/time"])
    tag_log = tag["dst"].ds.sel(time=time_slice)

    # Creating pandas series for xarrray dataset
    mean_lon_ = pd.Series(mean_df.geometry.x, name="longitude")
    mean_lat_ = pd.Series(mean_df.geometry.y, name="latitude")
    mode_lon_ = pd.Series(mode_df.geometry.x, name="longitude")
    mode_lat_ = pd.Series(mode_df.geometry.y, name="latitude")

    # Creating xarray datasets
    mean_coords = xr.Dataset(pd.concat([mean_lon_, mean_lat_], axis=1))
    mode_coords = xr.Dataset(pd.concat([mode_lon_, mode_lat_], axis=1))

    # Assigning dataarrays to variables
    mean_lon = mean_coords["longitude"]
    mean_lat = mean_coords["latitude"]
    mode_lon = mode_coords["longitude"]
    mode_lat = mode_coords["latitude"]

    temp_plot = tag_log["temperature"].hvplot(
        color="Red", title="Temperature (°C)", grid=True, height=200, width=600
    )
    depth_plot = tag_log["pressure"].hvplot(
        color="Blue", title="Pressure (m)", grid=True, height=200, width=600
    )
    lon_plot = (
        mean_lat.hvplot(label="mean", clim=[mean_lat_.min(), mean_lat_.max()])
        * mode_lat.hvplot(label="mode", clim=[mode_lat_.min(), mean_lat_.max()])
    ).opts(height=200, width=600, show_grid=True, title="Fish latitude over time")
    lat_plot = (
        mean_lon.hvplot(label="mean", clim=[mean_lon_.min(), mean_lat_.max()])
        * mode_lon.hvplot(label="mode", clim=[mode_lon_.min(), mean_lat_.max()])
    ).opts(height=200, width=600, show_grid=True, title="Fish longitude over time")

    return (temp_plot + depth_plot + lon_plot + lat_plot).cols(1)


def plot_track(tag_id="CB_A11071"):
    trajectories = read_trajectories(
        track_modes, f"{scratch_root}/{tag_id}", storage_options, format="parquet"
    )

    # Converting the trajectories to pandas DataFrames to access data easily
    mean_df = trajectories.trajectories[0].df
    mode_df = trajectories.trajectories[1].df

    # Adding month data
    mean_df["month"] = mean_df.index.month
    mode_df["month"] = mode_df.index.month

    # Converting back to trajectories
    mean_traj = mpd.Trajectory(
        mean_df, traj_id=mean_df.traj_id.drop_duplicates().values[0]
    )
    mode_traj = mpd.Trajectory(
        mode_df, traj_id=mode_df.traj_id.drop_duplicates().values[0]
    )
    trajectories = mpd.TrajectoryCollection([mean_traj, mode_traj])

    traj_plots = [
        traj.hvplot(
            c="month",
            tiles="CartoLight",
            cmap="rainbow",
            title=traj.id,
            width=500,
            height=500,
        )
        for traj in trajectories.trajectories
    ]

    return hv.Layout(traj_plots).cols(1)


def plot_emission(tag_id="DK_A10531"):
    ## Might not work if dask involved or slider involved, I have to test
    emission = (
        xr.open_dataset(
            f"{scratch_root}/{tag_id}/combined.zarr",
            engine="zarr",
            chunks={},
            inline_array=True,
            storage_options=storage_options,
        )
        .rename_vars({"pdf": "emission"})
        .drop_vars(["final", "initial"])
    )

    states = xr.open_dataset(
        f"{scratch_root}/{tag_id}/states.zarr",
        engine="zarr",
        chunks={},
        inline_array=True,
        storage_options=storage_options,
    ).where(emission["mask"])

    data = xr.merge([states, emission.drop_vars(["mask"])])
    plot1 = visualization.plot_map(data["states"], bbox, cmap="cool").opts(
        height=300, width=600
    )
    plot2 = visualization.plot_map(data["emission"], bbox, cmap="cool").opts(
        height=300, width=600
    )
    plot = hv.Layout([plot1, plot2]).cols(1)

    return plot

In [ ]:
# Panel parameters

# Initalizing the widget for tag selection
tag_widget = pn.widgets.Select(name="tag_id", value="NO_A12710", options=tag_list)

# Binding widget with the plots

time_plot = pn.bind(plot_time_series, tag_id=tag_widget)
track_plot = pn.bind(plot_track, tag_id=tag_widget)
# emission_plot = pn.bind(plot_emission,tag_id=tag_widget)
track_emission = pn.Row(time_plot, track_plot)

# Combining plots with the widget
plots = pn.Row(tag_widget, track_emission)

pn.template.FastListTemplate(
    site="Tag data display",
    title="plots",
    main=plots,
).servable();

In [ ]:
plots